##Importarea librariilor necesare

In [4]:
!pip install numpy



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install opencv-python



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2 
import numpy as np

In [2]:
def read_video(video_name):
	cap = cv2.VideoCapture(video_name)
	if (cap.isOpened() == False):
		print("error")
	else:
		total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
	return total_frames, cap
	

In [3]:

def get_frame(cap, total_frame : int, step : int):
	frames = []
	for frame_index in range(0, total_frame, step):
		cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
		ret, frame = cap.read()
	    
		if not ret:
			break
		
		frames.append(frame)
	return frames
		




In [ ]:
def detect_car_center(gray, prev_gray):
    diff = cv2.absdiff(gray, prev_gray)
    _, thresh = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    best_contour = None
    best_area = 0

    for c in contours:
        area = cv2.contourArea(c)
        if area > best_area:
            best_area = area
            best_contour = c

    if best_contour is None:
        return None 

    x, y, w, h = cv2.boundingRect(best_contour)
    cx = x + w // 2
    cy = y + h // 2

    return (cx, cy)


In [5]:

def smooth_coords(coords, window=5):
    xs = np.array([c[0] for c in coords], dtype=float)
    ys = np.array([c[1] for c in coords], dtype=float)

    kernel = np.ones(window) / window

    xs_smooth = np.convolve(xs, kernel, mode='same')
    ys_smooth = np.convolve(ys, kernel, mode='same')

    return list(zip(xs_smooth, ys_smooth))

In [8]:
def draw_traectory(coords_smooth, save_path=None):
    img = np.zeros((400, 400), dtype=np.uint8)
    pts = np.array([[int(x), int(y)] for (x, y) in coords_smooth], dtype=np.int32)

    cv2.polylines(img, [pts], isClosed=False, color=255, thickness=6)

    
    if save_path is not None:
        cv2.imwrite(save_path, img)

    # Afișare
    cv2.imshow("Trajectory", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
import os
folder_path = "videos"
output_folder = "trajectory_images"
for file_name in os.listdir(folder_path):
	if file_name.endswith("mp4"):
		video_path = os.path.join(folder_path, file_name)

		total_frames, cap = read_video(video_path)
		frames = get_frame(cap, total_frames, step = 10)
		gray_frames = []
		for frame in frames:
			gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
			gray_frames.append(gray)
		coords = []
		for i in range(1, len(gray_frames)):
			center = detect_car_center(gray_frames[i], gray_frames[i-1])
			coords.append(center)
		coords_smooth = smooth_coords([c for c in coords if c is not None], window=5)

		base_name = file_name.split(".")[0]
		save_path = os.path.join(output_folder, f"{base_name}_trajectory.png")
	
		draw_traectory(coords_smooth, save_path)
